In [ ]:
JOB_KEYWORDS = {
    "data_analyst": "data-analyst",
    "data_scientist": "data-scientist",
    "ml_engineer": "machine-learning-engineer"
}

def build_search_url(keyword):
    return f"https://www.vietnamworks.com/viec-lam?q={keyword}&sorting=relevant"

def scroll_to_bottom(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height



In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

options = webdriver.ChromeOptions()
# options.add_argument("--headless=new")
# options.add_argument("--disable-blink-features=AutomationControlled")
# options.add_argument("--window-size=1920,1080")
# options.add_argument("--disable-gpu")
# options.add_argument("--no-sandbox")
# options.add_argument(
#     "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
#     "AppleWebKit/537.36 (KHTML, like Gecko) "
#     "Chrome/120.0.0.0 Safari/537.36"
# )

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

all_job_links = []

for category, keyword in JOB_KEYWORDS.items():
    print(f"\n Crawl lĩnh vực: {category}")
    driver.get(build_search_url(keyword))

    page = 1

    while True:
        url = f"{build_search_url(keyword)}&page={page}"
        print("Load:", url)
        driver.get(url)
        time.sleep(3)

        scroll_to_bottom(driver)

        soup = BeautifulSoup(driver.page_source, "html.parser")
        jobs = soup.select("h2 a[href*='-jv']")

        if not jobs:
            print(" Không còn job, dừng")
            break

        print(f"Page {page}: {len(jobs)} jobs")

        for job in jobs:
            href = job.get("href")
            if href:
                all_job_links.append({
                    "category": category,
                    "url": href if href.startswith("http")
                        else "https://www.vietnamworks.com" + href
                })

        page += 1

#  remove duplicate
all_job_links = list({j["url"]: j for j in all_job_links}.values())

#  SAVE CSV
df = pd.DataFrame(all_job_links)
df.to_csv("vietnamworks_job_links.csv", index=False, encoding="utf-8-sig")

print(f"\n Tổng job links: {len(df)}")
driver.quit()



 Crawl lĩnh vực: data_analyst
Load: https://www.vietnamworks.com/viec-lam?q=data-analyst&sorting=relevant&page=1
Page 1: 50 jobs
Load: https://www.vietnamworks.com/viec-lam?q=data-analyst&sorting=relevant&page=2
Page 2: 50 jobs
Load: https://www.vietnamworks.com/viec-lam?q=data-analyst&sorting=relevant&page=3
Page 3: 50 jobs
Load: https://www.vietnamworks.com/viec-lam?q=data-analyst&sorting=relevant&page=4
Page 4: 50 jobs
Load: https://www.vietnamworks.com/viec-lam?q=data-analyst&sorting=relevant&page=5
Page 5: 50 jobs
Load: https://www.vietnamworks.com/viec-lam?q=data-analyst&sorting=relevant&page=6
Page 6: 50 jobs
Load: https://www.vietnamworks.com/viec-lam?q=data-analyst&sorting=relevant&page=7
Page 7: 50 jobs
Load: https://www.vietnamworks.com/viec-lam?q=data-analyst&sorting=relevant&page=8
Page 8: 50 jobs
Load: https://www.vietnamworks.com/viec-lam?q=data-analyst&sorting=relevant&page=9
Page 9: 3 jobs
Load: https://www.vietnamworks.com/viec-lam?q=data-analyst&sorting=relevant&pag

# Chạy từ đây 

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

df_links = pd.read_csv("vietnamworks_job_links.csv")
all_job_links = df_links.to_dict("records")



In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

def create_driver(headless=True):
    options = webdriver.ChromeOptions()

    if headless:
        options.add_argument("--headless=new")

    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/122.0.0.0 Safari/537.36"
    )

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    driver.set_page_load_timeout(30)
    return driver

def get_css_text(css):
    try:
        return driver.find_element(By.CSS_SELECTOR, css).text.strip()
    except EC:
        return ""

In [3]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
import random

def parse_job_detail(driver, job, retry=3):

    def get_xpath_text(xpath):
        try:
            return driver.find_element(By.XPATH, xpath).text.strip()
        except NoSuchElementException:
            return ""

    for attempt in range(retry):
        try:
            driver.get(job["url"])
            time.sleep(random.uniform(3, 5))

            return {
                "job_category": job["category"],
                "url": job["url"],

                # ===== BASIC =====
                "job_title": get_xpath_text("//h1"),

                # ===== COMPANY =====
                "company": get_xpath_text(
                    '//*[@id="vnwLayout__col"]/div/div[1]/div[2]/a'
                ),

                # ===== LOCATION =====
                "location": get_xpath_text(
                    '//*[@id="vnwLayout__col"]/div/p'
                ),

                # ===== JOB META =====
                # "job_type": get_css_text(
                #     "#vnwLayout__col > div > div.sc-7bf5461f-2.JtIju > p"
                # ),

                "date_posted": get_xpath_text(
                    '//*[@id="vnwLayout__col"]/div/div[2]/p'
                ),

                "skills": get_xpath_text(
                    '/html/body/main/div/main/div[2]/div/div/div/div[1]/div/div[1]/div/div[5]/div/div[4]/div/div[2]/p'
                ),

                "salary": get_xpath_text(
                    '//*[@id="vnwLayout__col"]/span'
                ),

                # ===== DESCRIPTION =====
                "job_description": get_xpath_text(
                    '//*[@id="vnwLayout__col"]/div/div[3]/div'
                ),

                # ===== REQUIREMENTS =====
                "requirements": get_xpath_text(
                    '//*[@id="vnwLayout__col"]/div/div[3]/div/div/div[2]'
                ),

                "company_field": get_xpath_text(
                    '//*[@id="vnwLayout__col"]/div/div[2]/p/span'
                ),

                # "working_form": get_xpath_text(
                #     '/html/body/main/div/main/div[2]/div/div/div/div[1]/div/div[1]/div/div[5]/div/div[14]/div/div[2]/p'
                # ),
            }

        except Exception as e:
            print(f" Lỗi attempt {attempt+1}/{retry}: {e}")
            time.sleep(5)

    return None


In [4]:
import pandas as pd
import time
import random
import os

OUTPUT_FILE = "vietnamworks_jobs_detail.csv"

all_results = []

BATCH_SIZE = 20        # restart driver
SAVE_EVERY = 50        # lưu CSV mỗi 50 job

driver = create_driver(headless=True)

for i, job in enumerate(all_job_links, start=1):
    print(f" Crawl {i}/{len(all_job_links)}: {job['url']}")

    result = parse_job_detail(driver, job)
    if result:
        all_results.append(result)

    # nghỉ random để tránh block
    time.sleep(random.uniform(1.5, 3))

    #  LƯU CSV MỖI 50 JOB
    if i % SAVE_EVERY == 0 and all_results:
        df = pd.DataFrame(all_results)

        # nếu file chưa tồn tại → ghi header
        write_header = not os.path.exists(OUTPUT_FILE)

        df.to_csv(
            OUTPUT_FILE,
            mode="a",
            header=write_header,
            index=False,
            encoding="utf-8-sig"
        )

        print(f" Đã lưu {i} job vào CSV")

        all_results.clear()  # giải phóng RAM

    #  RESTART DRIVER MỖI BATCH
    if i % BATCH_SIZE == 0:
        print(" Restart driver...")
        driver.quit()
        time.sleep(5)
        driver = create_driver(headless=True)

driver.quit()

#  LƯU PHẦN CÒN DƯ (nếu có)
if all_results:
    df = pd.DataFrame(all_results)
    write_header = not os.path.exists(OUTPUT_FILE)

    df.to_csv(
        OUTPUT_FILE,
        mode="a",
        header=write_header,
        index=False,
        encoding="utf-8-sig"
    )

print(" Hoàn tất crawl detail")


 Crawl 1/198: https://www.vietnamworks.com/chuyen-vien-phat-trien-du-an-djien-gio-1994066-jv?source=searchResults&searchType=2&placement=1994366&sortBy=relevance
 Crawl 2/198: https://www.vietnamworks.com/marketing-intern-content-and-design-1985840-jv?source=searchResults&searchType=2&placement=1986140&sortBy=relevance
 Crawl 3/198: https://www.vietnamworks.com/truong-phong-cong-nghe-1994140-jv?source=searchResults&searchType=2&placement=1994440&sortBy=relevance
 Crawl 4/198: https://www.vietnamworks.com/senior-product-business-planner--1986199-jv?source=searchResults&searchType=2&placement=1986499&sortBy=relevance
 Crawl 5/198: https://www.vietnamworks.com/chuyen-vien-marketing-1987337-jv?source=searchResults&searchType=2&placement=1987637&sortBy=relevance
 Crawl 6/198: https://www.vietnamworks.com/dye-lab-assistant-manager-1976095-jv?source=searchResults&searchType=2&placement=1976395&sortBy=relevance
 Crawl 7/198: https://www.vietnamworks.com/nhan-vien-ke-hoach--1985331-jv?source=se